In [1]:
!python -V

Python 3.11.3


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [47]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')

df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
print(f'std(duration) = {df["duration"].std():.2f}')

n_before = len(df)
df = df[(df.duration >= 1) & (df.duration <= 60)]
print(f'Remaining records: {len(df)/n_before*100:.0f}%')

categorical = ['PULocationID', 'DOLocationID']
numerical = []

df[categorical] = df[categorical].astype(str)

std(duration) = 46.45
Remaining records: 98%


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2421440 entries, 0 to 2463930
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [15]:
df.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
count,2.421440e+06,2421440,2421440,2.354481e+06,2.421440e+06,2.354481e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.421440e+06,2.354481e+06,2.354481e+06,2.421440e+06
mean,1.700020e+00,2022-01-17 01:35:16.982017792,2022-01-17 01:47:57.258748160,1.389813e+00,5.374377e+00,1.359217e+00,1.189319e+00,1.266161e+01,1.014453e+00,4.940617e-01,2.371678e+00,3.636815e-01,2.974866e-01,1.888460e+01,2.301668e+00,8.057827e-02,1.267128e+01
min,1.000000e+00,2008-12-31 23:03:01,2008-12-31 23:06:56,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-3.350000e+02,-4.500000e+00,-5.000000e-01,-1.252200e+02,-3.140000e+01,-3.000000e-01,-2.533000e+02,-2.500000e+00,-1.250000e+00,1.000000e+00
25%,1.000000e+00,2022-01-09 15:35:25,2022-01-09 15:47:44,1.000000e+00,1.060000e+00,1.000000e+00,1.000000e+00,6.500000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,3.000000e-01,1.130000e+01,2.500000e+00,0.000000e+00,6.433333e+00
50%,2.000000e+00,2022-01-17 11:58:39.500000,2022-01-17 12:09:32.500000,1.000000e+00,1.760000e+00,1.000000e+00,1.000000e+00,9.000000e+00,5.000000e-01,5.000000e-01,2.000000e+00,0.000000e+00,3.000000e-01,1.463000e+01,2.500000e+00,0.000000e+00,1.023333e+01
75%,2.000000e+00,2022-01-24 13:30:43.500000,2022-01-24 13:43:12.500000,1.000000e+00,3.120000e+00,1.000000e+00,1.000000e+00,1.350000e+01,2.500000e+00,5.000000e-01,3.000000e+00,0.000000e+00,3.000000e-01,1.980000e+01,2.500000e+00,0.000000e+00,1.613333e+01
max,6.000000e+00,2022-05-18 20:41:57,2022-05-18 20:47:45,9.000000e+00,3.061593e+05,9.900000e+01,5.000000e+00,4.010923e+05,3.350000e+01,3.300000e+00,8.888800e+02,1.933000e+02,3.000000e-01,4.010956e+05,2.500000e+00,1.250000e+00,6.000000e+01
std,4.705126e-01,NaN,NaN,9.836160e-01,5.524458e+02,5.585660e+00,4.876703e-01,2.579900e+02,1.234236e+00,7.060493e-02,2.697403e+00,1.637246e+00,3.859704e-02,2.581265e+02,7.097402e-01,3.084765e-01,8.999282e+00


In [39]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [40]:
df_train = read_dataframe('./data/yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [42]:
categorical = ['PULocationID', 'DOLocationID']
numerical = []

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [43]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [52]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_val = lr.predict(X_val)

print(f'RMSE train:{mean_squared_error(y_train, y_pred_train, squared=False):.2f}')
print(f'RMSE val:{mean_squared_error(y_val, y_pred_val, squared=False):.2f}')

RMSE train:6.99
RMSE val:7.79


In [53]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)